In [36]:
import requests
import urllib.request
import re
import argparse
import csv
from bs4 import BeautifulSoup
from itertools import zip_longest
#from selenium import webdriver
from lxml import html, etree


class MoviesToday ():
    
  
        
    def _download_web (url):
        
        reply = urllib.request.Request(url, data = None, headers = {
        'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.70 Safari/537.36'
        })
        response = urllib.request.urlopen(url, timeout = 10)
        html = response.read()
        return html 
    
    

    def get_movies_info (url):
        
        # Utilizamos Beautiful Soup para realizar el scraping
      
        response = urllib.request.urlopen(url, timeout = 10)

        html = response.read()
        
        bs = BeautifulSoup(html, 'html.parser')

        table = bs.findAll('table', id='main-content-table')

        table = table [0]

        names = []
        directors = []
        casting = []
        genres = []
        ratings = []
        links = []

        # Obtenemos la información de interés que está en una tabla
        for row in table.find_all ('tr'):
    
            for cell in row.find_all ('td'):
        
                for title in cell.find_all ('div', {"class": "mc-right"}):
            
                    data =title.h3
                    data = data.get_text()
                    names.append(data)
            
            
                    director = title.find ('div', {"class": "director"})
                    director = director.get_text()
                    director = director.replace('\n', '')
                    directors.append(director)
            
            
                    cast = title.find ('div', {"class" : "cast"})
                    cast = cast.get_text()
                    cast = cast.replace('\n', '')
                    casting.append(cast)
            
            
                    genre = title.find ('a', {"class" : "genre"})
                    genre = genre.get_text()
                    genres.append (genre)
           
            
                    rating = title.find ('div', {"class" : "avg-rating"})
                    rating = rating.get_text()
                    ratings.append (rating)
                    
            for row in table.find_all ('tr'):
    
                for cell in row.find_all ('td'):
        
                    for title in cell.find_all ('div', {"class": "mc-right"}):
            
                        trailer = title.find ('a')
                        link = trailer.get('href')
                        links.append(link)


        info = [names, directors, casting, genres, ratings, links]
        export_data = zip_longest(*info, fillvalue= '')
        
        print ('scraping successful')
        return export_data
        
        

    def _download_images (html):
        
        page = requests.get(url)
        soup = BeautifulSoup (page.content)

        images = []


        for row in soup.find_all ('tr'):
    
            for cell in row.find_all ('td'):

                for img in cell.find_all('div', {"class": "mc-left"}):
            
                    image = img.find('img')
                    image = image.get('src')
                    images.append(image)        

            
        for image in images:
    
            filename = image.split("/")[-1]
    
            rawImage = requests.get(image, stream = True)
    
            with open (filename, 'wb') as fd:
        
                for chunk in rawImage.iter_content(chunk_size = 1024):
                    fd.write(chunk)
    
        print ('Images downloaded')

    
    
    def data2csv (export_data):
        
        with open ('cinema.csv', 'w', newline = '') as r:
            wr = csv.writer(r)
            wr.writerow(('Names', 'Directors', 'Cast', 'Genre', 'Rating', 'Link'))
            wr.writerows(export_data)
        r.close()
        
        return 'csv created'

    
    
url = 'https://www.filmaffinity.com/uk/topcat.php?id=new_th_uk'    
    
MoviesToday();
MoviesToday._download_web(url);
#MoviesToday._download_images (html);
MoviesToday.data2csv (MoviesToday.get_movies_info (url));

scraping successful
